In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout, GRU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re
import codecs
import string

Using TensorFlow backend.


In [2]:
path = filename = "transcripts/friends_transcript.txt"
with codecs.open(path, encoding='utf-8') as f:
    text = f.read().lower()
    text = ''.join(filter(string.printable.__contains__,text))
    text = re.sub(r'[\r\n{}#"_`]',' ',text.strip())
    text = text.strip()

In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 54


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1261555


In [5]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(LSTM(128,input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
# filename = "friends_weights/weights-improvement-00-1.0237-full_data.hdf5"
# model.load_weights(filename)
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [7]:
filepath="friends_weights/weights-improvement-{epoch:02d}-{loss:.4f}-full_data.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=1200,epochs=1,callbacks=callbacks_list)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.1,0.2, 0.5, 0.6,1.0]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
1261555/1261555 [==============================] - 545s - loss: 1.0391   

----- diversity: 0.1
----- Generating with seed: "o'   s and... ross:    i'm a four? assis"
o'   s and... ross:    i'm a four? assistant:

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


   oh, i dont know what that i was going to be a little bit down the bathroom.                                        right i was a good time to the couch of the movie. rachel:  oh, i dont know. i mean, i dont know, i don't know what that i was going to be a little bit back.    rachel: oh, i don't know what that i was a good time to me.    ross: well, i dont know what that i was going to go t

----- diversity: 0.2
----- Generating with seed: "o'   s and... ross:    i'm a four? assis"
o'   s and... ross:    i'm a four? assistant:   thats a lot of the party and i was gonna be a little bit the best bathroom. rachel:  what? rachel:  oh, i don't know. i mean it's a different trip to the street with the baby and the man and i was all of the time. ross:  oh my god!                                        can the mailman.  rachel:  oh my god! i dont know. i mean it was a second? joey:  what? rachel:  oh my god!   rachel:  oh

----- diversity: 0.5
----- Generating with seed: "o'   s and... ross: